# producing $p(z | photometry)$ for ELAsTiCC

_Alex Malz (GCCL@RUB)_

The goal here is to generate mock photo-$z$ posteriors for host galaxies. 
These ones will be presumed Gaussian, with errors following the DESC SRD -- self-consistent, but not realistically complex.

In [1]:
import corner
import numpy as np
import pandas as pd

import scipy.stats as sps

In [2]:
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'DejaVu Serif'
# mpl.rcParams['axes.titlesize'] = 16
# mpl.rcParams['axes.labelsize'] = 14
# mpl.rcParams['savefig.dpi'] = 250
# mpl.rcParams['savefig.format'] = 'pdf'
# mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

## the hostlibs

In [3]:
hl_heads = {'SNIa': 18,
            'SNII': 18, 
            'SNIbc': 18, 
            'UNMATCHED_KN_SHIFT': 18,
            'UNMATCHED_COSMODC2': 17}

Let's pick one hostlib for now.

In [166]:
pick_one = 4

In [167]:
which_hl = list(hl_heads.keys())[pick_one]
hl_path = '/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/'+which_hl+'_GHOST.HOSTLIB.gz'
df = pd.read_csv(hl_path, skiprows=hl_heads[which_hl], delimiter=' ', header=0)

In [168]:
# zgrid = np.logspace(-3., np.log10(3.), 300)

In [169]:
nhost = len(df)
# nhost = 100
# df = df[:nhost]
# print(len(df))

In [170]:
# mini = True
# weirdo_galid = 10443470676

## forecasting-level photo-z

first create likelihood

In [171]:
sigma = 0.02

In [172]:
true_locs = df['ZTRUE'].values.reshape((nhost, 1))

In [173]:
# sps.norm(loc=[[0.], [1.], [2.]], scale=1).pdf([0.5,1.,1.5,2])
likelihood = sps.truncnorm(0., 3., loc=true_locs, scale=sigma*(1.+true_locs))

then draw point estimate

In [174]:
# np.shape(likelihood.pdf(zgrid))
obs_locs = likelihood.rvs()

then make forecasting posterior

In [175]:
posterior = sps.truncnorm(0., 3., loc=obs_locs, scale=sigma*(1.+obs_locs))

evaluate quantiles

In [176]:
quants = np.linspace(0., 1., 11)
# quants[0] += 0.001
# quants[-1] -= 0.001
# print(quants)

In [177]:
df[['Z0', 'Z10', 'Z20', 'Z30', 'Z40', 'Z50', 'Z60', 'Z70', 'Z80', 'Z90', 'Z100']] = posterior.ppf(quants)

In [178]:
df['P50'] = posterior.pdf(posterior.median())

In [179]:
df['IQR'] = posterior.ppf(0.75) - posterior.ppf(0.25)

save

In [180]:
df.to_csv('/global/cfs/cdirs/lsst/groups/TD/SN/SNANA/SURVEYS/LSST/ROOT/PLASTICC_DEV/HOSTLIB/zquants/'+which_hl+'_dummy_pz.csv')

## reconstructing a pdf from quantiles

### using "endpoints"

### using $p(z_{median})$

## manual quantiles from pzflow model

In [ ]:
# def quant_to_pdf(zgrid, qvals, zvals, zanch):
    
# def pdf_to_quant():